# Examen Clasificación 
## Ana Sofía Hinojosa Bale

In [198]:
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RBF
import warnings
from scipy import stats
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, cross_val_score
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
warnings.filterwarnings("ignore")

In [199]:
color1 = ['palevioletred', 'maroon', 'darkslategrey']
color2 = ['cadetblue', 'darkcyan', 'darkslategrey']
color3 = ['mediumpurple', 'rebeccapurple', 'darkslategrey']
color4 = ['steelblue', 'royalblue', 'darkslategrey']


def iteracion(funcion, X, Y, Z, x, y, z, n_iter=5, tipo="reducir incertidumbre", color=color1, nivel_conf=0.95):
    X = np.asarray(X).reshape(-1, 1)
    Y = np.asarray(Y).reshape(-1, 1)
    Z = np.asarray(Z).reshape(-1, 1)
    x = np.asarray(x).reshape(-1, 1)
    y = np.asarray(y).reshape(-1, 1)
    z = np.asarray(z).reshape(-1, 1)
    y = funcion(X, Y, Z).ravel()

    kernel = 1 * RBF(length_scale=1.0)
    gp = GPR(kernel=kernel, n_restarts_optimizer=10)
    gp.fit(X, y)

    for i in range(n_iter):
        y_pred, std = gp.predict(x, return_std=True)
        y_pred = y_pred.ravel()
        nivel_confianza = nivel_conf
        nc = norm.ppf(1 - (1 - nivel_confianza) / 2)
        # nc = 2.575
        y_upper = y_pred + nc * std
        y_lower = y_pred - nc * std

        if tipo == "reducir incertidumbre":
            index = np.argmax(std)
        elif tipo == "maximizar":
            index = np.argmax(y_upper)
        elif tipo == "minimizar":
            index = np.argmin(y_lower)
        x_val = x[index].reshape(1, -1)
        y_val = funcion(x_val).ravel()[0]

        X = np.vstack([X, x_val])
        y = np.concatenate([y, np.array([y_val])])

        gp.fit(X, y)

        plt.figure(figsize=(7, 5))
        plt.plot(x.ravel(), y_pred, c=color[0], lw=2, label="Predicción GP")
        plt.plot(x.ravel(), funcion(x).ravel(),
                 c="lightgray", ls="--", label="Función real")

        if X.shape[0] > 1:
            plt.scatter(X[:-1].ravel(), funcion(X[:-1]).ravel(),
                        c=color[1], label="Datos anteriores")

        plt.scatter(X[-1].ravel(), y_val, c=color[2], label="Nuevo punto")

        plt.plot(x.ravel(), y_upper, ls=":", lw=1,
                 color=color[0], label=f"IC {nivel_confianza*100:.0f}%, z={nc:.2f}")
        plt.plot(x.ravel(), y_lower, ls=":", lw=1, color=color[0])
        plt.fill_between(x.ravel(), y_lower, y_upper,
                         alpha=0.2, color=color[0])

        plt.grid(linestyle=":", alpha=0.7)
        plt.suptitle(f"Regresión con proceso gaussiano - Iteración {i+1}")
        plt.title(f"Selección de nuevo punto con criterio '{tipo}'")
        plt.xlabel("x")
        plt.ylabel("f(x)")
        plt.legend()
        plt.show()

In [200]:
'''def f(x, y, z) :
    return (6*x -2)**2 * np.sin(12*x -4) + (6*y -2)**2 * np.cos(12*y -4) + (6*z -2)**2 * np.sin(12*z -4)
np.random.seed(10)
X = np.random.uniform(0, 1, 3).reshape([-1, 1])
Y = np.random.uniform(0, 1, 3).reshape([-1, 1])
Z = np.random.uniform(0, 1, 3).reshape([-1, 1]) 
x = np.linspace(0, 1, 1000).reshape([-1, 1])
y = np.linspace(0, 1, 1000).reshape([-1, 1])
z = np.linspace(0, 1, 1000).reshape([-1, 1])

iteracion(f, X, Y, Z, x, y, z, n_iter=30)'''

'def f(x, y, z) :\n    return (6*x -2)**2 * np.sin(12*x -4) + (6*y -2)**2 * np.cos(12*y -4) + (6*z -2)**2 * np.sin(12*z -4)\nnp.random.seed(10)\nX = np.random.uniform(0, 1, 3).reshape([-1, 1])\nY = np.random.uniform(0, 1, 3).reshape([-1, 1])\nZ = np.random.uniform(0, 1, 3).reshape([-1, 1]) \nx = np.linspace(0, 1, 1000).reshape([-1, 1])\ny = np.linspace(0, 1, 1000).reshape([-1, 1])\nz = np.linspace(0, 1, 1000).reshape([-1, 1])\n\niteracion(f, X, Y, Z, x, y, z, n_iter=30)'

# Pregunta 2
Valor a predecir: average_rating>=4.3 -> 1, < 4.3 -> 0

In [201]:
data = pd.read_csv('adidas.csv')
data = data.drop(columns=['url', 'name', 'sku', 'description', 'images', 'source_website', 'source', 'breadcrumbs', 'language', 'currency', 'color', 'crawled_at'])
data['original_price'] = data['original_price'].str.replace('$', '').astype(float)
data['average_rating'] = data['average_rating'].apply(lambda x: 1 if x >= 4.3 else 0)
num_col = ['selling_price', 'original_price', 'reviews_count']
cat_col = ['availability', 'category', 'brand', 'country']
data = data.dropna().reset_index(drop=True)
data


,selling_price,original_price,availability,category,brand,country,average_rating,reviews_count
0,20,25.0,InStock,Clothing,adidas,USA,1,116
1,20,25.0,InStock,Clothing,adidas,USA,1,116
2,20,25.0,InStock,Clothing,adidas,USA,1,116
3,48,80.0,InStock,Clothing,adidas,USA,0,144
4,64,80.0,InStock,Shoes,adidas,USA,1,160
...,...,...,...,...,...,...,...,...
824,72,120.0,InStock,Shoes,adidas,USA,1,151
825,70,100.0,InStock,Shoes,adidas,USA,1,135
826,35,50.0,InStock,Shoes,adidas,USA,1,190
827,40,50.0,InStock,Shoes,adidas,USA,1,190


In [202]:
X = data.drop(columns=['average_rating'])
X = pd.get_dummies(X, columns=cat_col, drop_first=True)
y = data['average_rating']
x_test, x_train, y_test, y_train = train_test_split(X, y, train_size=0.7, random_state=42)


In [203]:
x_test

,selling_price,original_price,reviews_count,availability_OutOfStock,category_Clothing,category_Shoes
145,18,30.0,53,False,True,False
306,32,35.0,1352,False,True,False
234,24,30.0,78,False,True,False
220,28,35.0,7291,False,False,True
819,44,55.0,177,False,False,True
...,...,...,...,...,...,...
71,48,60.0,214,False,True,False
106,28,35.0,3812,False,False,True
270,52,65.0,671,False,True,False
435,44,55.0,17,False,False,True


In [204]:
scaler = StandardScaler().fit(x_train[num_col])
x_train[num_col] = scaler.transform(x_train[num_col])
x_test[num_col] = scaler.transform(x_test[num_col])

In [205]:
regresion = LogisticRegression()
regresion.fit(x_train,y_train)
y_pred = regresion.predict_proba(x_train)[:, 1]
roc_auc = roc_auc_score(y_train, y_pred)
roc_auc

0.7683724509426701

In [206]:
y_pred_test = regresion.predict_proba(x_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_pred_test)
roc_auc_test

0.6790495049504951

In [207]:
n = x_train.shape[0]
p = x_train.shape[1]
n,p

(249, 6)

In [208]:
b_0

array([3.03231624])

In [209]:
b_0[0]

np.float64(3.0323162415112948)

In [210]:
'''n = x_train.shape[0]
p = x_train.shape[1]
RSS = np.sum((y_pred_test - y_test)**2)
var = RSS / (n - p - 1)

b_0 = regresion.intercept_
b_i = regresion.coef_

X = np.column_stack((np.ones(n), x_train))
X = X.astype(float)
var_beta = np.linalg.inv(X.T @ X + np.eye(X.shape[1])*1e-6) * var
std_beta = np.sqrt(np.diag(var_beta))
t_stats = np.zeros(len(std_beta))
for i in range(len(std_beta)):
    t_stats[i] = (b_0[0] - b_i[0][i]) / std_beta[i]
p_values = [2 * (1 - stats.t.cdf(np.abs(t), n - p - 1)) for t in t_stats]'''


'n = x_train.shape[0]\np = x_train.shape[1]\nRSS = np.sum((y_pred_test - y_test)**2)\nvar = RSS / (n - p - 1)\n\nb_0 = regresion.intercept_\nb_i = regresion.coef_\n\nX = np.column_stack((np.ones(n), x_train))\nX = X.astype(float)\nvar_beta = np.linalg.inv(X.T @ X + np.eye(X.shape[1])*1e-6) * var\nstd_beta = np.sqrt(np.diag(var_beta))\nt_stats = np.zeros(len(std_beta))\nfor i in range(len(std_beta)):\n    t_stats[i] = (b_0[0] - b_i[0][i]) / std_beta[i]\np_values = [2 * (1 - stats.t.cdf(np.abs(t), n - p - 1)) for t in t_stats]'

# Pregunta 3

In [211]:
df = pd.read_csv('diabetes.csv')
y = df['Outcome']
X = df.drop(columns=['Outcome'])
scaler = StandardScaler().fit(X)
X = pd.DataFrame(scaler.transform(X), columns=X.columns)
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496
...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,0.870031,0.115169,-0.908682,2.532136
764,-0.547919,0.034598,0.046245,0.405445,-0.692891,0.610154,-0.398282,-0.531023
765,0.342981,0.003301,0.149641,0.154533,0.279594,-0.735190,-0.685193,-0.275760
766,-0.844885,0.159787,-0.470732,-1.288212,-0.692891,-0.240205,-0.371101,1.170732


In [212]:
def run_model(x: np.ndarray, y: np.ndarray, C: float) -> dict:
    kf = KFold(n_splits=10, shuffle=True)

    lr_model = LogisticRegression(max_iter=1000, C=C)
    lr_model.fit(x, y)
    lr_scores = cross_val_score(lr_model, x, y, cv=kf, scoring='precision')
    lr_predict = lr_model.predict_proba(x)[:, 1]
    lr_results = {
        'mean': lr_scores.mean(),
        'std': lr_scores.std(),
        'predictions': lr_predict
    }


    print(
        f"\nLogistic Regression: Precision = {lr_results['mean']:.4f} ± {lr_results['std']:.4f}")

    return {
        'lr_results': lr_results
    }

In [213]:
run_model(X.values, y.values, 2)


Logistic Regression: Precision = 0.7200 ± 0.1076


{'lr_results': {'mean': np.float64(0.7200367322735743),
  'std': np.float64(0.10757181350095558),
  'predictions': array([0.71981638, 0.04933123, 0.79421539, 0.04226898, 0.90041553,
         0.14753243, 0.06731905, 0.64097778, 0.7097593 , 0.03721347,
         0.22044378, 0.89587106, 0.78299976, 0.63136585, 0.62706761,
         0.39906913, 0.37177202, 0.19736995, 0.35667009, 0.23497608,
         0.39290264, 0.31760973, 0.93916912, 0.29478265, 0.70111706,
         0.44118653, 0.7353875 , 0.04651918, 0.5392435 , 0.27861723,
         0.42608398, 0.57257665, 0.04967327, 0.03712914, 0.43217669,
         0.15065195, 0.66342646, 0.39336248, 0.17298955, 0.5723509 ,
         0.74155082, 0.69469157, 0.11492119, 0.92606855, 0.62715685,
         0.95124578, 0.43321492, 0.04038956, 0.38030842, 0.03940934,
         0.03682523, 0.08594148, 0.06815446, 0.82610491, 0.70780091,
         0.02307931, 0.8817425 , 0.35816004, 0.83150367, 0.18399248,
         0.01011081, 0.52045455, 0.02385955, 0.3085604 , 0.

In [220]:
def run_model(x: np.ndarray, y: np.ndarray) -> dict:
    kf = KFold(n_splits=10, shuffle=True)

    svm_results = {}
    for kernel in kernels:
        svm_model = SVC(kernel=kernel, probability=True, max_iter=1000)
        scores = cross_val_score(svm_model, x, y, cv=kf, scoring='precision')
        svm_results[kernel] = {
            'mean': scores.mean(),
            'std': scores.std()
        }

    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(x, y)
    lr_scores = cross_val_score(lr_model, x, y, cv=kf, scoring='precision')
    lr_predict = lr_model.predict_proba(x)[:, 1]
    lr_results = {
        'mean': lr_scores.mean(),
        'std': lr_scores.std(),
        'predictions': lr_predict
    }

    print("SVM Results:")
    for kernel, result in svm_results.items():
        print(f"{kernel}: precision = {result['mean']:.4f} ± {result['std']:.4f}")
    print(
        f"\nLogistic Regression: precision = {lr_results['mean']:.4f} ± {lr_results['std']:.4f}")

    return {
        'svm_results': svm_results,
        'lr_results': lr_results
    }

In [221]:
kernels = ['linear', 'rbf', 'poly']
run_model(X.values, y.values)

SVM Results:
linear: precision = 0.7057 ± 0.1294
rbf: precision = 0.7184 ± 0.1299
poly: precision = 0.7605 ± 0.0986

Logistic Regression: precision = 0.7298 ± 0.0872


{'svm_results': {'linear': {'mean': np.float64(0.7057385762385762),
   'std': np.float64(0.12938284080075063)},
  'rbf': {'mean': np.float64(0.7183552738970695),
   'std': np.float64(0.1298844407014)},
  'poly': {'mean': np.float64(0.7605013368983957),
   'std': np.float64(0.09857943553197901)}},
 'lr_results': {'mean': np.float64(0.7297658529781292),
  'std': np.float64(0.08721060921784003),
  'predictions': array([0.71788557, 0.05004217, 0.7916649 , 0.04292874, 0.89864691,
         0.14842183, 0.06808818, 0.63750014, 0.70997257, 0.03807967,
         0.22122976, 0.89393557, 0.78146623, 0.6337848 , 0.62645407,
         0.39738602, 0.37235677, 0.19810911, 0.35598437, 0.23576997,
         0.39330902, 0.3185104 , 0.93786761, 0.29480507, 0.70010361,
         0.44101334, 0.73329128, 0.04727481, 0.53929577, 0.27939413,
         0.42647917, 0.5716599 , 0.05034582, 0.03780712, 0.43187454,
         0.15193868, 0.66127902, 0.39334987, 0.1735713 , 0.57246524,
         0.73887433, 0.6926571 , 0.11